# 🎬 Neural Network Visual Animations

This notebook provides **beautiful HTML/CSS visual animations** for neural networks:

## 🎨 Visual Features:
1. **🧠 Network Visualization** - Animated neurons with gradient colors and glowing effects
2. **📊 Training Charts** - Real-time loss graphs with SVG visualization  
3. **🎯 No Line Art** - Professional gradient backgrounds and smooth animations

## 🚀 Animations Included:
- **Forward Propagation**: Watch data flow through the network with glowing neurons
- **Training Progress**: See loss decrease with animated charts and progress bars
- **Interactive**: Step-by-step visualization with clear explanations

## ⚡ Key Features:
- Beautiful gradient backgrounds
- Smooth CSS animations and transitions
- Real-time value updates
- Progress tracking
- Professional modern design
- Pure Python + HTML/CSS (no external dependencies)

## Setup

In [74]:
import math
import random
import time
from IPython.display import HTML, display, clear_output

# Set random seed for reproducibility
random.seed(42)

print("🎬 Visual animation setup complete!")
print("📱 HTML/CSS animations for neural networks!")

🎬 Visual animation setup complete!
📱 HTML/CSS animations for neural networks!


## Neural Network Class

In [75]:
class AnimatedNetwork:
    def __init__(self):
        # Simple 2-3-1 network (using lists instead of numpy)
        self.W1 = [[0.5, -0.3], [0.2, 0.7], [-0.4, 0.6]]
        self.b1 = [0.1, 0.2, -0.1]
        self.W2 = [0.8, -0.5, 0.3]
        self.b2 = 0.2
        
        # Input and target
        self.X = [0.8, 0.3]
        self.y = 1.0
        
    def relu(self, z):
        if isinstance(z, list):
            return [max(0, val) for val in z]
        return max(0, z)
    
    def sigmoid(self, z):
        if isinstance(z, list):
            return [1 / (1 + math.exp(-max(-500, min(500, val)))) for val in z]
        return 1 / (1 + math.exp(-max(-500, min(500, z))))
    
    def matrix_multiply(self, matrix, vector):
        """Simple matrix-vector multiplication"""
        result = []
        for i in range(len(matrix)):
            sum_val = 0
            for j in range(len(vector)):
                sum_val += matrix[i][j] * vector[j]
            result.append(sum_val)
        return result
    
    def vector_add(self, vec1, vec2):
        """Add vector to each element or add bias"""
        if isinstance(vec2, list):
            return [vec1[i] + vec2[i] for i in range(len(vec1))]
        else:
            return [val + vec2 for val in vec1]
    
    def dot_product(self, vec1, vec2):
        """Calculate dot product"""
        return sum(vec1[i] * vec2[i] for i in range(len(vec1)))
    
    def forward_step_by_step(self):
        """Return each step of forward propagation"""
        steps = {}
        
        # Step 1: Input
        steps['input'] = {'values': self.X, 'description': 'Input values'}
        
        # Step 2: Hidden layer linear
        Z1_temp = self.matrix_multiply(self.W1, self.X)
        Z1 = self.vector_add(Z1_temp, self.b1)
        steps['hidden_linear'] = {'values': Z1, 'description': 'Z¹ = W¹X + b¹'}
        
        # Step 3: Hidden layer activation
        A1 = self.relu(Z1)
        steps['hidden_activation'] = {'values': A1, 'description': 'A¹ = ReLU(Z¹)'}
        
        # Step 4: Output linear
        Z2_temp = self.dot_product(self.W2, A1)
        Z2 = Z2_temp + self.b2
        steps['output_linear'] = {'values': Z2, 'description': 'Z² = W²A¹ + b²'}
        
        # Step 5: Output activation
        A2 = self.sigmoid(Z2)
        steps['output_activation'] = {'values': A2, 'description': 'A² = σ(Z²)'}
        
        # Step 6: Loss
        loss = -(self.y * math.log(A2 + 1e-8) + (1 - self.y) * math.log(1 - A2 + 1e-8))
        steps['loss'] = {'values': loss, 'description': f'Loss = {loss:.4f}'}
        
        return steps, A1, A2
    
    def backward_step_by_step(self, A1, A2):
        """Return each step of backward propagation"""
        steps = {}
        
        # Step 1: Output gradient
        dA2 = -(self.y / (A2 + 1e-8)) + (1 - self.y) / (1 - A2 + 1e-8)
        steps['output_grad'] = {'values': dA2, 'description': 'dL/dA² (output gradient)'}
        
        # Step 2: Output layer backward
        dZ2 = dA2 * A2 * (1 - A2)
        steps['output_backward'] = {'values': dZ2, 'description': 'dL/dZ² = dA² × σ\'(Z²)'}
        
        # Step 3: Hidden gradient
        dA1 = [self.W2[i] * dZ2 for i in range(len(self.W2))]
        steps['hidden_grad'] = {'values': dA1, 'description': 'dL/dA¹ = W²ᵀ × dZ²'}
        
        # Step 4: Hidden layer backward (simplified)
        Z1_temp = self.matrix_multiply(self.W1, self.X)
        Z1 = self.vector_add(Z1_temp, self.b1)
        dZ1 = [dA1[i] if Z1[i] > 0 else 0 for i in range(len(dA1))]
        steps['hidden_backward'] = {'values': dZ1, 'description': 'dL/dZ¹ = dA¹ × ReLU\'(Z¹)'}
        
        # Step 5: Weight gradients (simplified)
        dW2 = [dZ2 * A1[i] for i in range(len(A1))]
        dW1 = [[dZ1[i] * self.X[j] for j in range(len(self.X))] for i in range(len(dZ1))]
        steps['weight_grads'] = {
            'values': {'dW2': dW2, 'dW1': dW1}, 
            'description': 'Weight gradients computed'
        }
        
        return steps

# Create network
net = AnimatedNetwork()
print("🧠 Network created successfully!")

🧠 Network created successfully!


## Animation 1: Forward Propagation Steps

In [76]:
def create_network_visualization(input_vals, hidden_vals, output_val, 
                                active_layer=None, step_name="", description=""):
    """Create an HTML visualization of the neural network"""
    
    # Determine colors based on active layer
    input_color = "#4CAF50" if active_layer == "input" else "#E0E0E0"
    hidden_color = "#2196F3" if active_layer == "hidden" else "#E0E0E0"
    output_color = "#FF9800" if active_layer == "output" else "#E0E0E0"
    
    if active_layer == "input":
        input_glow = "0 0 20px #4CAF50"
        hidden_glow = "none"
        output_glow = "none"
    elif active_layer == "hidden":
        input_glow = "none"
        hidden_glow = "0 0 20px #2196F3"
        output_glow = "none"
    elif active_layer == "output":
        input_glow = "none"
        hidden_glow = "none"
        output_glow = "0 0 20px #FF9800"
    else:
        input_glow = hidden_glow = output_glow = "none"
    
    html = f"""
    <style>
        @keyframes pulse {{
            0% {{ transform: scale(1); }}
            50% {{ transform: scale(1.1); }}
            100% {{ transform: scale(1); }}
        }}
        
        @keyframes flow {{
            0% {{ opacity: 0; transform: translateX(0); }}
            50% {{ opacity: 1; }}
            100% {{ opacity: 0; transform: translateX(100%); }}
        }}
        
        .network-container {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            border-radius: 20px;
            padding: 40px;
            margin: 20px auto;
            max-width: 900px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
        }}
        
        .network-title {{
            color: white;
            font-size: 28px;
            font-weight: bold;
            text-align: center;
            margin-bottom: 10px;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        }}
        
        .network-subtitle {{
            color: rgba(255,255,255,0.9);
            font-size: 18px;
            text-align: center;
            margin-bottom: 30px;
        }}
        
        .network {{
            display: flex;
            justify-content: space-around;
            align-items: center;
            height: 400px;
            position: relative;
        }}
        
        .layer {{
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            gap: 30px;
            z-index: 10;
        }}
        
        .neuron {{
            width: 80px;
            height: 80px;
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            color: white;
            font-weight: bold;
            font-size: 14px;
            transition: all 0.3s ease;
            box-shadow: 0 5px 15px rgba(0,0,0,0.3);
            position: relative;
            cursor: pointer;
        }}
        
        .neuron:hover {{
            transform: scale(1.2);
        }}
        
        .neuron.active {{
            animation: pulse 1s infinite;
        }}
        
        .neuron-value {{
            position: absolute;
            bottom: -25px;
            font-size: 12px;
            color: white;
            background: rgba(0,0,0,0.5);
            padding: 2px 8px;
            border-radius: 10px;
        }}
        
        .layer-label {{
            color: white;
            font-weight: bold;
            margin-top: 20px;
            font-size: 16px;
            text-shadow: 1px 1px 2px rgba(0,0,0,0.3);
        }}
        
        .info-panel {{
            background: rgba(255,255,255,0.1);
            border-radius: 15px;
            padding: 20px;
            margin-top: 30px;
            color: white;
        }}
        
        .info-title {{
            font-size: 20px;
            font-weight: bold;
            margin-bottom: 10px;
        }}
        
        .info-content {{
            font-size: 14px;
            line-height: 1.6;
        }}
    </style>
    
    <div class="network-container">
        <div class="network-title">🧠 Neural Network Visualization</div>
        <div class="network-subtitle">{step_name}</div>
        
        <div class="network">
            <!-- Input Layer -->
            <div class="layer">
                <div class="neuron {'active' if active_layer == 'input' else ''}" 
                     style="background: {input_color}; box-shadow: {input_glow};">
                    X₁
                    <div class="neuron-value">{input_vals[0]:.3f}</div>
                </div>
                <div class="neuron {'active' if active_layer == 'input' else ''}"
                     style="background: {input_color}; box-shadow: {input_glow};">
                    X₂
                    <div class="neuron-value">{input_vals[1]:.3f}</div>
                </div>
                <div class="layer-label">INPUT</div>
            </div>
            
            <!-- Hidden Layer -->
            <div class="layer">
                <div class="neuron {'active' if active_layer == 'hidden' else ''}"
                     style="background: {hidden_color}; box-shadow: {hidden_glow};">
                    H₁
                    <div class="neuron-value">{hidden_vals[0] if hidden_vals else '?'}</div>
                </div>
                <div class="neuron {'active' if active_layer == 'hidden' else ''}"
                     style="background: {hidden_color}; box-shadow: {hidden_glow};">
                    H₂
                    <div class="neuron-value">{hidden_vals[1] if hidden_vals else '?'}</div>
                </div>
                <div class="neuron {'active' if active_layer == 'hidden' else ''}"
                     style="background: {hidden_color}; box-shadow: {hidden_glow};">
                    H₃
                    <div class="neuron-value">{hidden_vals[2] if hidden_vals else '?'}</div>
                </div>
                <div class="layer-label">HIDDEN</div>
            </div>
            
            <!-- Output Layer -->
            <div class="layer">
                <div class="neuron {'active' if active_layer == 'output' else ''}"
                     style="background: {output_color}; box-shadow: {output_glow};">
                    Y
                    <div class="neuron-value">{output_val if output_val else '?'}</div>
                </div>
                <div class="layer-label">OUTPUT</div>
            </div>
            
            <!-- Connections (simplified SVG) -->
            <svg style="position: absolute; width: 100%; height: 100%; top: 0; left: 0; pointer-events: none;">
                <!-- Input to Hidden connections -->
                <line x1="25%" y1="35%" x2="50%" y2="25%" 
                      stroke="{'#FFD700' if active_layer == 'hidden' else 'rgba(255,255,255,0.3)'}" 
                      stroke-width="{'3' if active_layer == 'hidden' else '2'}" opacity="0.7"/>
                <line x1="25%" y1="35%" x2="50%" y2="50%" 
                      stroke="{'#FFD700' if active_layer == 'hidden' else 'rgba(255,255,255,0.3)'}" 
                      stroke-width="{'3' if active_layer == 'hidden' else '2'}" opacity="0.7"/>
                <line x1="25%" y1="35%" x2="50%" y2="75%" 
                      stroke="{'#FFD700' if active_layer == 'hidden' else 'rgba(255,255,255,0.3)'}" 
                      stroke-width="{'3' if active_layer == 'hidden' else '2'}" opacity="0.7"/>
                      
                <line x1="25%" y1="65%" x2="50%" y2="25%" 
                      stroke="{'#FFD700' if active_layer == 'hidden' else 'rgba(255,255,255,0.3)'}" 
                      stroke-width="{'3' if active_layer == 'hidden' else '2'}" opacity="0.7"/>
                <line x1="25%" y1="65%" x2="50%" y2="50%" 
                      stroke="{'#FFD700' if active_layer == 'hidden' else 'rgba(255,255,255,0.3)'}" 
                      stroke-width="{'3' if active_layer == 'hidden' else '2'}" opacity="0.7"/>
                <line x1="25%" y1="65%" x2="50%" y2="75%" 
                      stroke="{'#FFD700' if active_layer == 'hidden' else 'rgba(255,255,255,0.3)'}" 
                      stroke-width="{'3' if active_layer == 'hidden' else '2'}" opacity="0.7"/>
                
                <!-- Hidden to Output connections -->
                <line x1="50%" y1="25%" x2="75%" y2="50%" 
                      stroke="{'#FFD700' if active_layer == 'output' else 'rgba(255,255,255,0.3)'}" 
                      stroke-width="{'3' if active_layer == 'output' else '2'}" opacity="0.7"/>
                <line x1="50%" y1="50%" x2="75%" y2="50%" 
                      stroke="{'#FFD700' if active_layer == 'output' else 'rgba(255,255,255,0.3)'}" 
                      stroke-width="{'3' if active_layer == 'output' else '2'}" opacity="0.7"/>
                <line x1="50%" y1="75%" x2="75%" y2="50%" 
                      stroke="{'#FFD700' if active_layer == 'output' else 'rgba(255,255,255,0.3)'}" 
                      stroke-width="{'3' if active_layer == 'output' else '2'}" opacity="0.7"/>
            </svg>
        </div>
        
        <div class="info-panel">
            <div class="info-title">Current Operation</div>
            <div class="info-content">{description}</div>
        </div>
    </div>
    """
    
    return html

In [61]:
def animate_forward_propagation():
    """Show animated forward propagation with HTML visuals"""
    
    # Get forward propagation steps
    steps, A1, A2 = net.forward_step_by_step()
    
    animations = [
        ("input", "Step 1: Input Layer", f"Loading input values: X₁={net.X[0]:.3f}, X₂={net.X[1]:.3f}", None, None),
        ("hidden", "Step 2: Hidden Linear Transform", f"Computing Z = W·X + b for hidden layer", None, None),
        ("hidden", "Step 3: Hidden Activation", f"Applying ReLU activation: A = max(0, Z)", A1, None),
        ("output", "Step 4: Output Linear Transform", f"Computing Z = W·A + b for output layer", A1, None),
        ("output", "Step 5: Output Activation", f"Applying Sigmoid: Y = 1/(1+e^(-Z))", A1, A2),
        (None, "Step 6: Loss Calculation", f"Loss = {steps['loss']['values']:.4f}", A1, A2)
    ]
    
    for active_layer, title, description, hidden, output in animations:
        clear_output(wait=True)
        
        # Format values for display
        if hidden is not None:
            hidden_display = [f"{h:.3f}" for h in hidden]
        else:
            hidden_display = None
        
        if output is not None:
            output_display = f"{output:.4f}"
        else:
            output_display = None
            
        html = create_network_visualization(
            net.X, 
            hidden_display,
            output_display,
            active_layer,
            title,
            description
        )
        display(HTML(html))
        time.sleep(2)
    
    # Final summary
    clear_output(wait=True)
    summary_html = f"""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                border-radius: 20px; padding: 30px; margin: 20px auto; 
                max-width: 700px; color: white; text-align: center;">
        <h2>✅ Forward Propagation Complete!</h2>
        <div style="display: flex; justify-content: space-around; margin-top: 20px;">
            <div>
                <div style="font-size: 24px; font-weight: bold;">{A2:.4f}</div>
                <div style="font-size: 14px; opacity: 0.9;">Prediction</div>
            </div>
            <div>
                <div style="font-size: 24px; font-weight: bold;">{net.y}</div>
                <div style="font-size: 14px; opacity: 0.9;">Target</div>
            </div>
            <div>
                <div style="font-size: 24px; font-weight: bold;">{steps['loss']['values']:.4f}</div>
                <div style="font-size: 14px; opacity: 0.9;">Loss</div>
            </div>
        </div>
    </div>
    """
    display(HTML(summary_html))
    
    return steps, A1, A2

# Run the visual forward propagation
print("🎬 Starting visual forward propagation...")
forward_steps, A1, A2 = animate_forward_propagation()

## Animation 2: Backward Propagation Steps

In [ ]:
def create_gradient_visualization(input_vals, hidden_vals, output_val, 
                                 gradients, active_step="", step_name="", description=""):
    """Create HTML visualization for gradient flow"""
    
    # Determine gradient colors based on magnitude
    def get_gradient_color(grad_val):
        if abs(grad_val) > 0.1:
            return "#FF4444"  # High gradient - bright red
        elif abs(grad_val) > 0.05:
            return "#FF8800"  # Medium gradient - orange
        elif abs(grad_val) > 0.01:
            return "#FFBB00"  # Low gradient - yellow
        else:
            return "#888888"  # Very low gradient - gray
    
    # Get gradient values for display
    output_grad = gradients.get('output_grad', 0)
    hidden_grads = gradients.get('hidden_grads', [0, 0, 0])
    
    html = f"""
    <style>
        @keyframes gradient-pulse {{
            0% {{ opacity: 0.5; transform: scale(1); }}
            50% {{ opacity: 1; transform: scale(1.1); }}
            100% {{ opacity: 0.5; transform: scale(1); }}
        }}
        
        @keyframes gradient-flow {{
            0% {{ opacity: 0; }}
            25% {{ opacity: 0.7; }}
            75% {{ opacity: 0.7; }}
            100% {{ opacity: 0; }}
        }}
        
        @keyframes arrow-flow {{
            0% {{ stroke-dashoffset: 20; }}
            100% {{ stroke-dashoffset: 0; }}
        }}
        
        .gradient-container {{
            background: linear-gradient(135deg, #2c3e50 0%, #34495e 100%);
            border-radius: 20px;
            padding: 40px;
            margin: 20px auto;
            max-width: 900px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.4);
        }}
        
        .gradient-title {{
            color: #ecf0f1;
            font-size: 28px;
            font-weight: bold;
            text-align: center;
            margin-bottom: 10px;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);
        }}
        
        .gradient-subtitle {{
            color: #bdc3c7;
            font-size: 18px;
            text-align: center;
            margin-bottom: 30px;
        }}
        
        .network-gradient {{
            display: flex;
            justify-content: space-around;
            align-items: center;
            height: 400px;
            position: relative;
        }}
        
        .gradient-layer {{
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            gap: 30px;
            z-index: 10;
        }}
        
        .gradient-neuron {{
            width: 80px;
            height: 80px;
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            color: white;
            font-weight: bold;
            font-size: 14px;
            transition: all 0.3s ease;
            box-shadow: 0 5px 15px rgba(0,0,0,0.3);
            position: relative;
            border: 3px solid transparent;
        }}
        
        .gradient-neuron.active {{
            animation: gradient-pulse 2s infinite;
            border-color: #e74c3c;
            box-shadow: 0 0 30px rgba(231, 76, 60, 0.6);
        }}
        
        .gradient-value {{
            position: absolute;
            bottom: -35px;
            font-size: 11px;
            color: #ecf0f1;
            background: rgba(0,0,0,0.7);
            padding: 3px 8px;
            border-radius: 10px;
            white-space: nowrap;
        }}
        
        .gradient-arrow {{
            position: absolute;
            stroke: #e74c3c;
            stroke-width: 3;
            stroke-dasharray: 10,5;
            animation: arrow-flow 2s infinite;
            opacity: 0.8;
        }}
        
        .gradient-info {{
            background: rgba(236, 240, 241, 0.1);
            border-radius: 15px;
            padding: 20px;
            margin-top: 30px;
            color: #ecf0f1;
            border: 1px solid rgba(236, 240, 241, 0.2);
        }}
        
        .gradient-info-title {{
            font-size: 20px;
            font-weight: bold;
            margin-bottom: 10px;
            color: #e74c3c;
        }}
        
        .gradient-equation {{
            font-family: 'Courier New', monospace;
            background: rgba(0,0,0,0.3);
            padding: 10px;
            border-radius: 5px;
            margin: 10px 0;
            border-left: 4px solid #e74c3c;
        }}
    </style>
    
    <div class="gradient-container">
        <div class="gradient-title">🔄 Gradient Flow Visualization</div>
        <div class="gradient-subtitle">{step_name}</div>
        
        <div class="network-gradient">
            <!-- Output Layer -->
            <div class="gradient-layer">
                <div class="gradient-neuron {'active' if active_step == 'output' else ''}" 
                     style="background: {get_gradient_color(output_grad)};">
                    ∂L/∂Y
                    <div class="gradient-value">grad: {output_grad:.4f}</div>
                </div>
                <div style="color: #bdc3c7; font-weight: bold; margin-top: 15px;">OUTPUT</div>
            </div>
            
            <!-- Hidden Layer -->
            <div class="gradient-layer">
                <div class="gradient-neuron {'active' if active_step == 'hidden' else ''}"
                     style="background: {get_gradient_color(hidden_grads[0])};">
                    ∂L/∂H₁
                    <div class="gradient-value">grad: {hidden_grads[0]:.4f}</div>
                </div>
                <div class="gradient-neuron {'active' if active_step == 'hidden' else ''}"
                     style="background: {get_gradient_color(hidden_grads[1])};">
                    ∂L/∂H₂
                    <div class="gradient-value">grad: {hidden_grads[1]:.4f}</div>
                </div>
                <div class="gradient-neuron {'active' if active_step == 'hidden' else ''}"
                     style="background: {get_gradient_color(hidden_grads[2])};">
                    ∂L/∂H₃
                    <div class="gradient-value">grad: {hidden_grads[2]:.4f}</div>
                </div>
                <div style="color: #bdc3c7; font-weight: bold; margin-top: 15px;">HIDDEN</div>
            </div>
            
            <!-- Input Layer -->
            <div class="gradient-layer">
                <div class="gradient-neuron {'active' if active_step == 'weights' else ''}"
                     style="background: #7f8c8d;">
                    ∂L/∂W
                    <div class="gradient-value">weights</div>
                </div>
                <div class="gradient-neuron {'active' if active_step == 'weights' else ''}"
                     style="background: #7f8c8d;">
                    ∂L/∂W
                    <div class="gradient-value">gradients</div>
                </div>
                <div style="color: #bdc3c7; font-weight: bold; margin-top: 15px;">WEIGHTS</div>
            </div>
            
            <!-- Gradient Flow Arrows -->
            <svg style="position: absolute; width: 100%; height: 100%; top: 0; left: 0; pointer-events: none;">
                <!-- Output to Hidden arrows -->
                <defs>
                    <marker id="arrowhead" markerWidth="10" markerHeight="7" 
                            refX="0" refY="3.5" orient="auto">
                        <polygon points="0 0, 10 3.5, 0 7" fill="#e74c3c" />
                    </marker>
                </defs>
                
                <!-- Animated gradient flow arrows -->
                <line x1="75%" y1="50%" x2="50%" y2="25%" 
                      class="gradient-arrow" marker-end="url(#arrowhead)"
                      style="opacity: {'1' if active_step in ['output', 'hidden'] else '0.3'}"/>
                <line x1="75%" y1="50%" x2="50%" y2="50%" 
                      class="gradient-arrow" marker-end="url(#arrowhead)"
                      style="opacity: {'1' if active_step in ['output', 'hidden'] else '0.3'}"/>
                <line x1="75%" y1="50%" x2="50%" y2="75%" 
                      class="gradient-arrow" marker-end="url(#arrowhead)"
                      style="opacity: {'1' if active_step in ['output', 'hidden'] else '0.3'}"/>
                
                <!-- Hidden to Weights arrows -->
                <line x1="50%" y1="25%" x2="25%" y2="35%" 
                      class="gradient-arrow" marker-end="url(#arrowhead)"
                      style="opacity: {'1' if active_step == 'weights' else '0.3'}"/>
                <line x1="50%" y1="50%" x2="25%" y2="50%" 
                      class="gradient-arrow" marker-end="url(#arrowhead)"
                      style="opacity: {'1' if active_step == 'weights' else '0.3'}"/>
                <line x1="50%" y1="75%" x2="25%" y2="65%" 
                      class="gradient-arrow" marker-end="url(#arrowhead)"
                      style="opacity: {'1' if active_step == 'weights' else '0.3'}"/>
            </svg>
        </div>
        
        <div class="gradient-info">
            <div class="gradient-info-title">Current Step: {step_name}</div>
            <div>{description}</div>
            <div class="gradient-equation">
                Chain Rule: ∂L/∂w = ∂L/∂y × ∂y/∂z × ∂z/∂w
            </div>
        </div>
    </div>
    """
    
    return html

def animate_backward_propagation():
    """Show animated backward propagation with visual gradients"""
    
    # Get backward propagation steps
    steps = net.backward_step_by_step(A1, A2)
    
    animations = [
        ("output", "Step 1: Output Gradient", 
         "Computing gradient of loss with respect to output: ∂L/∂A²",
         {'output_grad': steps['output_grad']['values'], 'hidden_grads': [0, 0, 0]}),
        
        ("output", "Step 2: Output Layer Backward", 
         "Applying chain rule through sigmoid activation: ∂L/∂Z² = ∂L/∂A² × σ'(Z²)",
         {'output_grad': steps['output_backward']['values'], 'hidden_grads': [0, 0, 0]}),
        
        ("hidden", "Step 3: Hidden Layer Gradients", 
         "Propagating gradients to hidden layer: ∂L/∂A¹ = W²ᵀ × ∂L/∂Z²",
         {'output_grad': steps['output_backward']['values'], 
          'hidden_grads': steps['hidden_grad']['values']}),
        
        ("hidden", "Step 4: Hidden Layer Backward", 
         "Applying chain rule through ReLU: ∂L/∂Z¹ = ∂L/∂A¹ × ReLU'(Z¹)",
         {'output_grad': steps['output_backward']['values'], 
          'hidden_grads': steps['hidden_backward']['values']}),
        
        ("weights", "Step 5: Weight Gradients", 
         "Computing weight gradients: ∂L/∂W = ∂L/∂Z × Aᵀ (ready for optimization)",
         {'output_grad': steps['output_backward']['values'], 
          'hidden_grads': steps['hidden_backward']['values']})
    ]
    
    for active_step, title, description, gradient_data in animations:
        clear_output(wait=True)
        
        html = create_gradient_visualization(
            net.X, 
            A1,
            A2,
            gradient_data,
            active_step,
            title,
            description
        )
        display(HTML(html))
        time.sleep(3)
    
    # Final summary
    clear_output(wait=True)
    
    dW2 = steps['weight_grads']['values']['dW2']
    dW1 = steps['weight_grads']['values']['dW1']
    max_grad_W1 = max(max(abs(val) for val in row) for row in dW1)
    max_grad_W2 = max(abs(val) for val in dW2)
    
    summary_html = f"""
    <div style="background: linear-gradient(135deg, #27ae60 0%, #2ecc71 100%); 
                border-radius: 20px; padding: 30px; margin: 20px auto; 
                max-width: 700px; color: white; text-align: center;">
        <h2>✅ Backward Propagation Complete!</h2>
        <div style="display: flex; justify-content: space-around; margin-top: 20px;">
            <div>
                <div style="font-size: 20px; font-weight: bold;">{max_grad_W2:.6f}</div>
                <div style="font-size: 14px; opacity: 0.9;">Max W2 Gradient</div>
            </div>
            <div>
                <div style="font-size: 20px; font-weight: bold;">{max_grad_W1:.6f}</div>
                <div style="font-size: 14px; opacity: 0.9;">Max W1 Gradient</div>
            </div>
            <div>
                <div style="font-size: 20px; font-weight: bold;">{len(dW1) * len(dW1[0]) + len(dW2)}</div>
                <div style="font-size: 14px; opacity: 0.9;">Total Parameters</div>
            </div>
        </div>
        <div style="margin-top: 20px; font-size: 16px; opacity: 0.9;">
            🔄 All gradients computed and ready for weight updates!
        </div>
    </div>
    """
    display(HTML(summary_html))
    
    return steps

# Run backward propagation animation
print("🔄 Starting visual backward propagation...")
backward_steps = animate_backward_propagation()

## Animation 3: Training Progress

In [77]:
def animate_training_progress():
    """Show polished animated training simulation"""
    
    clear_output(wait=True)
    print("╔" + "═" * 58 + "╗")
    print("║" + " " * 12 + "🏃‍♂️ NEURAL NETWORK TRAINING" + " " * 17 + "║")
    print("╚" + "═" * 58 + "╝")
    
    learning_rate = 0.1
    epochs = 12
    
    print()
    print("┌────────────── TRAINING CONFIGURATION ──────────────┐")
    print(f"│  🎛️  Learning Rate: {learning_rate:<28} │")
    print(f"│  🔄 Epochs: {epochs:<35} │")
    print(f"│  🎯 Optimization: Gradient Descent              │")
    print(f"│  📊 Loss Function: Binary Cross-Entropy         │")
    print("└──────────────────────────────────────────────────┘")
    print()
    
    time.sleep(2)
    
    # Initialize training state
    current_loss = 2.0
    losses = []
    improvements = []
    
    for epoch in range(epochs):
        clear_output(wait=True)
        
        # Training header
        print("╔" + "═" * 58 + "╗")
        print("║" + " " * 12 + "🏃‍♂️ NEURAL NETWORK TRAINING" + " " * 17 + "║")
        print("╚" + "═" * 58 + "╝")
        print()
        
        # Epoch progress
        epoch_progress = "█" * (epoch + 1) + "░" * (epochs - epoch - 1)
        epoch_percent = int(((epoch + 1) / epochs) * 100)
        print(f"┌─ Epoch {epoch + 1:2d}/{epochs} ──── [{epoch_progress}] {epoch_percent:3d}% ─┐")
        
        # Simulate gradient descent step with realistic dynamics
        gradient = current_loss * 0.12 * (1 + 0.1 * random.uniform(-1, 1))
        current_loss -= learning_rate * gradient
        current_loss = max(0.01, current_loss)
        
        # Add training noise (more realistic)
        if epoch > 2:
            noise_factor = max(0.02 * (epochs - epoch) / epochs, 0.005)  # Decreasing noise
            noise = random.uniform(-noise_factor, noise_factor * 0.5)
            current_loss += noise
            current_loss = max(0.01, current_loss)
        
        losses.append(current_loss)
        
        # Calculate improvement
        improvement = losses[0] - current_loss if losses else 0
        improvement_percent = (improvement / losses[0]) * 100 if losses[0] > 0 else 0
        improvements.append(improvement_percent)
        
        # Current metrics display
        print(f"│                                                  │")
        print(f"│  📉 Current Loss: {current_loss:8.4f}                    │")
        print(f"│  📈 Gradient: {gradient:11.4f}                       │")
        print(f"│  🎯 Improvement: {improvement_percent:6.1f}%                     │")
        print(f"│                                                  │")
        
        # Visual loss bar
        loss_scale = min(int(current_loss * 25), 25)
        loss_visual = "█" * max(1, loss_scale) + "░" * (25 - max(1, loss_scale))
        print(f"│  Loss Visual: [{loss_visual}] │")
        print(f"│                                                  │")
        
        # Training status and trend analysis
        if epoch > 0:
            trend = losses[epoch] < losses[epoch-1]
            trend_icon = "📉" if trend else "📈"
            trend_text = "Decreasing" if trend else "Increasing"
            change = abs(losses[epoch] - losses[epoch-1])
            print(f"│  {trend_icon} Trend: {trend_text:<12} (Δ {change:.4f})        │")
        else:
            print(f"│  🚀 Status: Training initialized                │")
        
        # Performance assessment
        if current_loss < 0.05:
            status = "🎯 Excellent convergence!"
            status_color = "green"
        elif current_loss < 0.2:
            status = "✅ Good progress"
            status_color = "blue"
        elif current_loss < 0.5:
            status = "📈 Making progress"
            status_color = "orange"
        elif epoch < 3:
            status = "🚀 Starting to learn"
            status_color = "yellow"
        else:
            status = "⚠️  Slow convergence"
            status_color = "red"
        
        print(f"│  {status:<42} │")
        print("└──────────────────────────────────────────────────┘")
        print()
        
        # Weight update animation
        print("⚖️  Updating weights", end="")
        for j in range(4):
            weight_visual = ["⚡", "💫", "✨", "🔥"][j % 4]
            print(f"\r⚖️  Updating weights {weight_visual} ", end="")
            time.sleep(0.3)
        print("✅")
        print()
        
        # Mini loss history (last 5 epochs)
        if epoch >= 3:
            print("📊 Recent Loss History:")
            start_idx = max(0, epoch - 4)
            recent_losses = losses[start_idx:epoch+1]
            recent_epochs = list(range(start_idx + 1, epoch + 2))
            
            for i, (e, loss) in enumerate(zip(recent_epochs, recent_losses)):
                bar_length = int((1 - loss / losses[0]) * 15) if losses[0] > 0 else 0
                bar = "▓" * bar_length + "░" * (15 - bar_length)
                marker = "👈" if e == epoch + 1 else "  "
                print(f"   Epoch {e:2d}: [{bar}] {loss:.4f} {marker}")
            print()
        
        time.sleep(1.8)
    
    # Training completion summary
    clear_output(wait=True)
    print("╔" + "═" * 58 + "╗")
    print("║" + " " * 14 + "🎉 TRAINING COMPLETE!" + " " * 19 + "║")
    print("╠" + "═" * 58 + "╣")
    
    # Final metrics
    final_improvement = ((losses[0] - losses[-1]) / losses[0]) * 100
    convergence_rate = "Fast" if final_improvement > 80 else "Medium" if final_improvement > 60 else "Slow"
    
    print(f"║ Initial Loss: {losses[0]:6.4f} │ Final Loss: {losses[-1]:6.4f}      ║")
    print(f"║ Total Improvement: {final_improvement:5.1f}% │ Rate: {convergence_rate:<12}    ║")
    print(f"║ Epochs: {epochs:2d} │ Learning Rate: {learning_rate} │ Status: Success ✅ ║")
    print("╠" + "═" * 58 + "╣")
    
    # Complete loss history visualization
    print("║                    📈 LOSS CURVE                    ║")
    print("║                                                    ║")
    for i, loss in enumerate(losses):
        if i % 2 == 0 or i == len(losses) - 1:  # Show every other epoch + final
            bar_length = int((1 - loss / losses[0]) * 20) if losses[0] > 0 else 0
            bar = "▓" * bar_length + "░" * (20 - bar_length)
            print(f"║ Epoch {i+1:2d}: [{bar}] {loss:.4f}  ║")
    
    print("║                                                    ║")
    print("╚" + "═" * 58 + "╝")
    
    # Training insights
    print()
    print("🎯 Training Insights:")
    print(f"  • Network learned to reduce loss by {final_improvement:.1f}%")
    print(f"  • Convergence rate: {convergence_rate}")
    print(f"  • Final prediction accuracy: {(1 - losses[-1]) * 100:.1f}%")
    if final_improvement > 90:
        print("  • 🏆 Excellent training performance!")
    elif final_improvement > 70:
        print("  • ✅ Good training performance!")
    else:
        print("  • 📈 Consider adjusting hyperparameters")
    
    return losses

# Run polished training animation
print("🏃‍♂️ Starting polished training animation...")
training_losses = animate_training_progress()

╔══════════════════════════════════════════════════════════╗
║              🎉 TRAINING COMPLETE!                   ║
╠══════════════════════════════════════════════════════════╣
║ Initial Loss: 1.9753 │ Final Loss: 1.7258      ║
║ Total Improvement:  12.6% │ Rate: Slow            ║
║ Epochs: 12 │ Learning Rate: 0.1 │ Status: Success ✅ ║
╠══════════════════════════════════════════════════════════╣
║                    📈 LOSS CURVE                    ║
║                                                    ║
║ Epoch  1: [░░░░░░░░░░░░░░░░░░░░] 1.9753  ║
║ Epoch  3: [░░░░░░░░░░░░░░░░░░░░] 1.9315  ║
║ Epoch  5: [░░░░░░░░░░░░░░░░░░░░] 1.8919  ║
║ Epoch  7: [▓░░░░░░░░░░░░░░░░░░░] 1.8398  ║
║ Epoch  9: [▓░░░░░░░░░░░░░░░░░░░] 1.7891  ║
║ Epoch 11: [▓▓░░░░░░░░░░░░░░░░░░] 1.7436  ║
║ Epoch 12: [▓▓░░░░░░░░░░░░░░░░░░] 1.7258  ║
║                                                    ║
╚══════════════════════════════════════════════════════════╝

🎯 Training Insights:
  • Network learned to reduce loss by

## Animation 4: Weight Update Visualization

In [64]:
def animate_weight_updates():
    """Show polished animated weight update process"""
    
    clear_output(wait=True)
    print("╔" + "═" * 58 + "╗")
    print("║" + " " * 12 + "⚖️  WEIGHT OPTIMIZATION" + " " * 19 + "║")
    print("╚" + "═" * 58 + "╝")
    
    # Get gradients
    steps = net.backward_step_by_step(A1, A2)
    dW1 = steps['weight_grads']['values']['dW1']
    dW2 = steps['weight_grads']['values']['dW2']
    
    learning_rate = 0.1
    
    print()
    print("┌─────────── OPTIMIZATION CONFIGURATION ───────────┐")
    print(f"│  🎛️  Learning Rate: {learning_rate:<25} │")
    print(f"│  📐 Update Rule: W ← W - α∇W                 │")
    print(f"│  🔢 Parameters: {len(net.W1)*len(net.W1[0]) + len(net.W2):<26} │")
    print(f"│  ⚖️  Optimizer: Stochastic Gradient Descent     │")
    print("└───────────────────────────────────────────────────┘")
    print()
    time.sleep(2)
    
    phases = [
        ("Analyzing Current Weights", "📊"),
        ("Computing Weight Gradients", "🧮"), 
        ("Applying Gradient Updates", "⚡"),
        ("Validating New Weights", "✅")
    ]
    
    for phase_idx, (phase_name, phase_icon) in enumerate(phases):
        clear_output(wait=True)
        
        # Phase header
        print("╔" + "═" * 58 + "╗")
        print("║" + " " * 12 + "⚖️  WEIGHT OPTIMIZATION" + " " * 19 + "║")
        print("╠" + "═" * 58 + "╣")
        
        # Phase progress
        phase_progress = "█" * (phase_idx + 1) + "░" * (len(phases) - phase_idx - 1)
        phase_percent = int(((phase_idx + 1) / len(phases)) * 100)
        print(f"║ Phase: [{phase_progress}] {phase_percent}% │ {phase_icon} {phase_name} ║")
        print("╚" + "═" * 58 + "╝")
        print()
        
        if phase_idx == 0:  # Current weights analysis
            print("┌─────────────── CURRENT WEIGHT MATRICES ──────────────┐")
            print("│                                                       │")
            print("│  🔹 Layer 1 Weights (Input → Hidden):                │")
            print("│                                                       │")
            
            # Animated weight display
            for i, row in enumerate(net.W1):
                weight_display = ""
                for j in range(len(row)):
                    weight_display += f"{row[j]:7.3f}"
                    if j < len(row) - 1:
                        weight_display += " "
                    time.sleep(0.3)
                    print(f"\r│     Row {i+1}: [{weight_display:<15}]                        │", end="")
                print()
            
            print("│                                                       │")
            print("│  🔸 Layer 2 Weights (Hidden → Output):               │")
            print("│                                                       │")
            
            weight_display = ""
            for i, val in enumerate(net.W2):
                weight_display += f"{val:7.3f}"
                if i < len(net.W2) - 1:
                    weight_display += " "
                time.sleep(0.3)
                print(f"\r│     [{weight_display:<27}]                      │", end="")
            print()
            print("│                                                       │")
            print("└───────────────────────────────────────────────────────┘")
            
        elif phase_idx == 1:  # Gradient computation
            print("┌─────────────── COMPUTED GRADIENTS ───────────────────┐")
            print("│                                                       │")
            print("│  🔹 Layer 1 Gradients (dW1):                         │")
            print("│                                                       │")
            
            for i, row in enumerate(dW1):
                gradient_display = ""
                magnitude_total = 0
                for j, val in enumerate(row):
                    gradient_display += f"{val:8.4f}"
                    magnitude_total += abs(val)
                    if j < len(row) - 1:
                        gradient_display += " "
                    
                    # Color coding based on magnitude
                    if abs(val) > 0.1:
                        color_icon = "🔴"  # High
                    elif abs(val) > 0.05:
                        color_icon = "🟠"  # Medium
                    elif abs(val) > 0.01:
                        color_icon = "🟡"  # Low
                    else:
                        color_icon = "⚪"  # Very low
                    
                    time.sleep(0.4)
                    print(f"\r│  {color_icon} Row {i+1}: [{gradient_display:<17}] │{magnitude_total/len(row):.4f}│", end="")
                print()
            
            print("│                                                       │")
            print("│  🔸 Layer 2 Gradients (dW2):                         │")
            print("│                                                       │")
            
            gradient_display = ""
            for i, val in enumerate(dW2):
                gradient_display += f"{val:8.4f}"
                if i < len(dW2) - 1:
                    gradient_display += " "
                
                if abs(val) > 0.1:
                    color_icon = "🔴"
                elif abs(val) > 0.05:
                    color_icon = "🟠"
                elif abs(val) > 0.01:
                    color_icon = "🟡"
                else:
                    color_icon = "⚪"
                
                time.sleep(0.4)
                print(f"\r│  {color_icon} [{gradient_display:<30}]              │", end="")
            print()
            print("│                                                       │")
            print("└───────────────────────────────────────────────────────┘")
            
        elif phase_idx == 2:  # Weight updates
            print("┌─────────────── WEIGHT UPDATE PROCESS ────────────────┐")
            print("│                                                       │")
            print("│  Formula: W_new = W_old - α × gradient               │")
            print(f"│  Learning Rate (α): {learning_rate}                             │")
            print("│                                                       │")
            print("│  🔹 Updating Layer 1 Weights:                        │")
            
            W1_new = []
            for i in range(len(net.W1)):
                row_new = []
                print(f"│                                                       │")
                for j in range(len(net.W1[i])):
                    old_val = net.W1[i][j]
                    grad_val = dW1[i][j]
                    new_val = old_val - learning_rate * grad_val
                    row_new.append(new_val)
                    
                    change = abs(old_val - new_val)
                    direction = "↓" if old_val > new_val else "↑"
                    
                    # Animation of the calculation
                    calc_steps = [
                        f"  W1[{i+1},{j+1}]: {old_val:.3f}",
                        f"  W1[{i+1},{j+1}]: {old_val:.3f} - {learning_rate}×{grad_val:.3f}",
                        f"  W1[{i+1},{j+1}]: {new_val:.3f} {direction}{change:.3f}"
                    ]
                    
                    for step in calc_steps:
                        print(f"\r│{step:<55}│", end="")
                        time.sleep(0.6)
                    print()
                
                W1_new.append(row_new)
            
            print("│                                                       │")
            print("│  🔸 Updating Layer 2 Weights:                        │")
            print("│                                                       │")
            
            W2_new = []
            for i in range(len(net.W2)):
                old_val = net.W2[i]
                grad_val = dW2[i]
                new_val = old_val - learning_rate * grad_val
                W2_new.append(new_val)
                
                change = abs(old_val - new_val)
                direction = "↓" if old_val > new_val else "↑"
                
                calc_steps = [
                    f"  W2[{i+1}]: {old_val:.3f}",
                    f"  W2[{i+1}]: {old_val:.3f} - {learning_rate}×{grad_val:.3f}",
                    f"  W2[{i+1}]: {new_val:.3f} {direction}{change:.3f}"
                ]
                
                for step in calc_steps:
                    print(f"\r│{step:<55}│", end="")
                    time.sleep(0.6)
                print()
            
            print("│                                                       │")
            print("└───────────────────────────────────────────────────────┘")
            
        elif phase_idx == 3:  # Validation
            print("┌─────────────── UPDATE VALIDATION ────────────────────┐")
            print("│                                                       │")
            
            # Calculate statistics
            total_change_W1 = sum(sum(abs(net.W1[i][j] - W1_new[i][j]) for j in range(len(net.W1[i]))) for i in range(len(net.W1)))
            total_change_W2 = sum(abs(net.W2[i] - W2_new[i]) for i in range(len(net.W2)))
            
            max_change_W1 = max(max(abs(net.W1[i][j] - W1_new[i][j]) for j in range(len(net.W1[i]))) for i in range(len(net.W1)))
            max_change_W2 = max(abs(net.W2[i] - W2_new[i]) for i in range(len(net.W2)))
            
            print(f"│  📊 Layer 1 Statistics:                              │")
            print(f"│     • Total Change: {total_change_W1:7.4f}                      │")
            print(f"│     • Max Change: {max_change_W1:9.4f}                      │")
            print(f"│     • Parameters: {len(net.W1)*len(net.W1[0])}                             │")
            print("│                                                       │")
            print(f"│  📊 Layer 2 Statistics:                              │")
            print(f"│     • Total Change: {total_change_W2:7.4f}                      │")
            print(f"│     • Max Change: {max_change_W2:9.4f}                      │")
            print(f"│     • Parameters: {len(net.W2)}                             │")
            print("│                                                       │")
            
            # Impact assessment
            total_change = total_change_W1 + total_change_W2
            if total_change > 0.2:
                impact = "🚀 Significant - Strong learning signal!"
                impact_color = "green"
            elif total_change > 0.05:
                impact = "✅ Moderate - Good learning progress"
                impact_color = "blue"
            elif total_change > 0.01:
                impact = "📈 Small - Steady improvement"
                impact_color = "yellow"
            else:
                impact = "⚠️  Minimal - Consider higher learning rate"
                impact_color = "orange"
            
            print(f"│  🎯 Update Impact: {impact:<31} │")
            print("│                                                       │")
            print("└───────────────────────────────────────────────────────┘")
        
        time.sleep(2)
    
    # Final summary with before/after comparison
    clear_output(wait=True)
    print("╔" + "═" * 58 + "╗")
    print("║" + " " * 15 + "⚖️  OPTIMIZATION COMPLETE" + " " * 16 + "║")
    print("╠" + "═" * 58 + "╣")
    
    print("║                   BEFORE → AFTER                    ║")
    print("║                                                    ║")
    print("║ Layer 1:                                           ║")
    for i in range(len(net.W1)):
        old_str = f"[{net.W1[i][0]:6.3f}, {net.W1[i][1]:6.3f}]"
        new_str = f"[{W1_new[i][0]:6.3f}, {W1_new[i][1]:6.3f}]"
        print(f"║ Row {i+1}: {old_str} → {new_str}        ║")
    
    print("║                                                    ║")
    print("║ Layer 2:                                           ║")
    old_w2_str = f"[{net.W2[0]:5.3f}, {net.W2[1]:5.3f}, {net.W2[2]:5.3f}]"
    new_w2_str = f"[{W2_new[0]:5.3f}, {W2_new[1]:5.3f}, {W2_new[2]:5.3f}]"
    print(f"║ {old_w2_str} → {new_w2_str} ║")
    
    print("║                                                    ║")
    print(f"║ Total Change: {total_change:.4f} │ Status: Success ✅       ║")
    print("╚" + "═" * 58 + "╝")
    
    return W1_new, W2_new

# Run polished weight updates
print("⚖️ Starting polished weight update animation...")
new_W1, new_W2 = animate_weight_updates()

╔══════════════════════════════════════════════════════════╗
║               ⚖️  OPTIMIZATION COMPLETE                ║
╠══════════════════════════════════════════════════════════╣
║                   BEFORE → AFTER                    ║
║                                                    ║
║ Layer 1:                                           ║
║ Row 1: [ 0.500, -0.300] → [ 0.528, -0.289]        ║
║ Row 2: [ 0.200,  0.700] → [ 0.182,  0.693]        ║
║ Row 3: [-0.400,  0.600] → [-0.400,  0.600]        ║
║                                                    ║
║ Layer 2:                                           ║
║ [0.800, -0.500, 0.300] → [0.818, -0.475, 0.300] ║
║                                                    ║
║ Total Change: 0.1059 │ Status: Success ✅       ║
╚══════════════════════════════════════════════════════════╝


## Summary: Complete Network Visualization

In [65]:
def run_complete_animation():
    """Run complete animation sequence with visual HTML animations"""
    
    print("🎬 Starting complete animation sequence...")
    print("=" * 50)
    
    # Run forward propagation
    print("1. Forward Propagation")
    forward_steps, A1, A2 = animate_forward_propagation()
    
    print("\n2. Training Process")
    training_losses = animate_training_progress()
    
    print("\n✅ Animation sequence complete!")
    print(f"Final prediction: {A2:.4f}")
    print(f"Final training loss: {training_losses[-1]:.4f}")
    
    return forward_steps, training_losses

# Demo message
print("🎬 Neural Network Visual Animations Ready!")
print("\nAvailable animations:")
print("  • animate_forward_propagation() - Visual network animation")
print("  • animate_training_progress() - Training with loss charts")
print("  • run_complete_animation() - Full sequence")
print("\nRun any function to see the animations!")

🎬 Neural Network Visual Animations Ready!

Available animations:
  • animate_forward_propagation() - Visual network animation
  • animate_training_progress() - Training with loss charts
  • run_complete_animation() - Full sequence

Run any function to see the animations!
